# 1. Installations

In [ ]:
!pip install -U transformers
!pip install dspy-ai
!pip install weaviate-client

In [ ]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF")
# Login to Hugging Face
login(hf_token)

# 2. Téléchargement des données 

Ici les données sont simples et ne demandent pas de prétraitement comme le chunking

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # affichage des phrases enitières dans le dataset

df = pd.read_csv("/kaggle/input/competition.csv")

df

In [ ]:
# Ne prendre en compte que 100 données du dataset 
df = df[(df.HostSegmentTitle == "Featured") & (df.Subtitle.notna())][-100:]
df[['Title', 'Subtitle']].head()

In [14]:
filtered_df = df[df['Title'].str.contains("Bosch", case=False, na=False)]

print(filtered_df[['Title', 'Subtitle']])

                                  Title                       Subtitle
5135  Bosch Production Line Performance  Reduce manufacturing failures


# 3. Téléchargement des données dans Weaviate

In [ ]:
import weaviate
from weaviate.embedded import EmbeddedOptions
import re

# Connection à Weaviate client in embedded mode
client = weaviate.Client(embedded_options=EmbeddedOptions(),
                             additional_headers={
                                "X-Huggingface-Api-Key": hf_token,
                             }
                         )

# Définition du schéma de la collection de données
schema = {
   "classes": [
       {
           "class": "MyExampleIndex",
           "vectorizer": "text2vec-huggingface",
            "moduleConfig": {
                "text2vec-huggingface": {
                    "model": "sentence-transformers/all-MiniLM-L6-v2",
                }
            },
           "properties": [
               {
                   "name": "content", # This is a required property name to be able to use Weaviate as RM with DSPy
                   "dataType": ["text"]
               }
           ]
       }      
   ]
}

# Supprimer la collection de données existante si elle existe déjà à partir d'une exécution précédente
if client.schema.exists("MyExampleIndex"):
    client.schema.delete_class("MyExampleIndex")
client.schema.create(schema)

# Configuration de la taile du batch 
client.batch.configure(batch_size=100)

with client.batch as batch: 
    for _, row in df.iterrows():
        properties = {
            "content": row['Title'] + ' Kaggle Competition: ' + row['Subtitle']
        }
        batch.add_data_object(
            data_object=properties,
            class_name="MyExampleIndex"
        )

In [8]:
client.query.aggregate("MyExampleIndex").with_meta_count().do()

{'data': {'Aggregate': {'MyExampleIndex': [{'meta': {'count': 89}}]}}}

# 4. Recherche sémantique dans la base de données Weaviate 

In [ ]:
import json 

response = (
    client.query
    .get("MyExampleIndex", ["content"])
    .with_near_text({"concepts": ["e-commerce"]})
    .with_limit(3)
    .do()
)

print(json.dumps(response, indent=4))

# 5. Configuration de DSPy 

In [ ]:
!pip install --upgrade google-api-python-client
!pip install --upgrade protobuf

In [ ]:
import dspy

## Language Model: Google Gemma



In [ ]:
# Définition du llm via DSPY
llm = dspy.HFModel(model = 'google/gemma-2b') # dspy.OpenAI(model='gpt-3.5-turbo')

In [ ]:
example_query = "Which Kaggle competition should I look at to learn more about recommender systems in e-commerce?"

response = llm(example_query, do_sample=True) # do_sample = True active l'échantillionnage sinon cela se fait de façon déterministe
                                              # top_k , pour choisir le nbr de mots les plus probables qui peuvent tomber
print(response)

## Retriever Model: Weaviate
Dspy est lié à WeaviateRM : il faut donc reconfigurer notre Weaviate préalable pour pouvoir l'utiliser avec DSPY

In [ ]:
from dspy.retrieve.weaviate_rm import WeaviateRM

# Configuration du nouveau Retriever Model 
rm = WeaviateRM("MyExampleIndex", 
                weaviate_client = client)

In [ ]:
# Configuration de DSPY sur le llm et le rm
dspy.settings.configure(lm = llm, 
                        rm = rm)

# 6. Mise en place du RAG


## 6.1 Sans compilation (entraînement)

In [ ]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [ ]:
uncompiled_rag = RAG()

response = uncompiled_rag(example_query)

print(response.answer)

In [ ]:
llm.inspect_history(n=1)

## 6.2 Avec compilation (entraînement)

In [ ]:
example_question1 = "Has there been Kaggle competition about cute animals and if yes, which one?"
example_answer1 = "You might be interested in the PetFinder.my Adoption Prediction competition."

example_question2 = "I'm interested in autonomous driving. What Kaggle competition can you recommend?"
example_answer2 ="You might be interested in the Lyft Motion Prediction for Autonomous Vehicles or Lyft 3D Object Detection for Autonomous Vehicles competition,"

example_question3 = "What Kaggle competitions should I look at to learn more about predicting the stock market?"
example_answer3 = "You might be interested in the JPX Tokyo Stock Exchange Prediction or Jane Street Market Prediction or Ubiquant Market Prediction competition."

In [ ]:
trainset = [dspy.Example(question=example_question1, 
                         answer=example_answer1).with_inputs('question'),
            dspy.Example(question=example_question2, 
                         answer=example_answer2).with_inputs('question'),
           dspy.Example(question=example_question3, 
                         answer=example_answer3).with_inputs('question'),]
trainset

In [ ]:
!pip uninstall -y wandb

In [ ]:
from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric=dspy.evaluate.answer_exact_match)
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

In [ ]:
response = compiled_rag(example_query)

print(response.answer)